In [ ]:
import pandas as pd
import numpy as np
import sys

In [2]:
# Funções Utilitárias 

# Retorna o tamanho do objeto em MB
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

# Exibe as dimensões, o tamanho e os 5 principais registros de dataframe
def inspect_df(df_name, df):
    print('{0} --- dimensions: {1};  size: {2}'.format(df_name, df.shape, size(df)))  
    display(df.head())
    
# Exporta dataframe para CSV, o formato para carregar dados no Neo4j 
def export_to_csv(df, out):
    df.to_csv(out, sep='|', columns=df.columns, index=False)   

In [3]:
min_orders   = 5     # Contagem mínima de pedidos por usuário
sample_count = 100   # Número de usuários para selecionar aleatoriamente

# Carregar dados do conjunto de avaliação "prior" (consulte o dicionário de dados para obter a definição de "eval_set") 
order_user = pd.read_csv('instacart_2017_05_01/orders.csv')
order_user = order_user[order_user['eval_set'] == 'prior']

# Obter distribuição do número de pedidos por usuário
user_order_count = order_user.groupby('user_id').agg({'order_id':'count'}).rename(columns={'order_id':'num_orders'}).reset_index()
print('Distribution of number of orders per user:')
display(user_order_count['num_orders'].describe())

# Selecione usuários que compraram pelo menos 'min_orders'
user_order_atleast_x = user_order_count[user_order_count['num_orders'] >= min_orders]

# Para reprodutibilidade, defina a semente antes de fazer uma amostra aleatória
np.random.seed(1111)
user_sample = np.random.choice(user_order_atleast_x['user_id'], sample_count, replace=False)

# Subconjunto 'order_user' para incluir registros associados aos 100 usuários selecionados aleatoriamente
order_user = order_user[order_user['user_id'].isin(user_sample)]
order_user = order_user[['order_id','user_id','order_number','order_dow','order_hour_of_day']]
inspect_df('order_user', order_user)

Distribution of number of orders per user:


count    206209.000000
mean         15.590367
std          16.654774
min           3.000000
25%           5.000000
50%           9.000000
75%          19.000000
max          99.000000
Name: num_orders, dtype: float64

order_user --- dimensions: (1901, 5);  size: 0.09 MB


,order_id,user_id,order_number,order_dow,order_hour_of_day
11334,2808127,701,1,2,14
11335,2677145,701,2,3,11
11336,740361,701,3,1,13
11337,2866491,701,4,3,12
11338,1676999,701,5,4,11


In [4]:
# Carregar pedidos associados aos nossos 100 usuários selecionados, juntamente com os produtos contidos nesses pedidos
order_product = pd.read_csv('instacart_2017_05_01/order_products__prior.csv')
order_product = order_product[order_product['order_id'].isin(order_user.order_id.unique())][['order_id','product_id']]
inspect_df('order_product', order_product)

order_product --- dimensions: (19840, 2);  size: 0.48 MB


,order_id,product_id
1855,209,39409
1856,209,20842
1857,209,16965
1858,209,8021
1859,209,23001


In [5]:
# Carregar produtos comprados pelos nossos 100 usuários selecionados
products = pd.read_csv('instacart_2017_05_01/products.csv')
products = products[products['product_id'].isin(order_product.product_id.unique())]
inspect_df('products', products)

products --- dimensions: (3959, 4);  size: 0.46 MB


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
33,34,Peanut Butter Cereal,121,14
44,45,European Cucumber,83,4
98,99,Local Living Butter Lettuce,83,4
115,116,English Muffins,93,3


In [6]:
#products[products['product_id']==2506]#['product_name']
#products.set_value(36617,'product_name','')
#products.set_value(3237,'product_name','')
#products.set_value(40541,'product_name','')
#products.set_value(2001,'product_name','')
#products.set_value(24505,'product_name','')
#products.set_value(28030,'product_name','')
#products.set_value(31377,'product_name','')

#products = products.dropna(subset=['aisle_id'])
products.isnull().sum()

product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64

In [7]:
# Load entire aisle data as it contains the names related to the aisle IDs from the 'products' data
aisles = pd.read_csv('instacart_2017_05_01/aisles.csv')
inspect_df('aisles', aisles)


aisles --- dimensions: (134, 2);  size: 0.01 MB


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [8]:
# Load entire department data as it contains the names related to the department IDs from the 'products' data
departments = pd.read_csv('instacart_2017_05_01/departments.csv')
inspect_df('departments', departments)

departments --- dimensions: (21, 2);  size: 0.00 MB


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [17]:
#export_to_csv(order_user,    '~/neo4j_instacart/import/neo4j_order_user.csv')
#export_to_csv(order_product, '~/neo4j_instacart/import/neo4j_order_product.csv')    
export_to_csv(products,      '~/neo4j_instacart/import/neo4j_products.csv')
#export_to_csv(aisles,        '~/neo4j_instacart/import/neo4j_aisles.csv')
#export_to_csv(departments,   '~/neo4j_instacart/import/neo4j_departments.csv')

In [10]:
from py2neo import Graph

# Set up authentication parameters
#authenticate("localhost:7474", "neo4j", "rsf159753") 

# Connect to authenticated graph database
g = Graph("bolt://localhost:7687/", user="neo4j", password="123")

g.begin()

In [11]:
# Each time this notebook is run, we start with an empty graph database
g.run("MATCH (n) DETACH DELETE n;")    

# We drop and recreate our node constraints
#g.run("DROP CONSTRAINT ON (order:Order)             ASSERT order.order_id            IS UNIQUE;")
#g.run("DROP CONSTRAINT ON (user:User)               ASSERT user.user_id              IS UNIQUE;")
#g.run("DROP CONSTRAINT ON (product:Product)         ASSERT product.product_id        IS UNIQUE;")
#g.run("DROP CONSTRAINT ON (aisle:Aisle)             ASSERT aisle.aisle_id            IS UNIQUE;")
#g.run("DROP CONSTRAINT ON (department:Department)   ASSERT department.department_id  IS UNIQUE;")

g.run("CREATE CONSTRAINT ON (order:Order)           ASSERT order.order_id            IS UNIQUE;")
g.run("CREATE CONSTRAINT ON (user:User)             ASSERT user.user_id              IS UNIQUE;")
g.run("CREATE CONSTRAINT ON (product:Product)       ASSERT product.product_id        IS UNIQUE;")
g.run("CREATE CONSTRAINT ON (aisle:Aisle)           ASSERT aisle.aisle_id            IS UNIQUE;")
g.run("CREATE CONSTRAINT ON (department:Department) ASSERT department.department_id  IS UNIQUE;")


In [20]:
#Carrega dados de produtos para o Neo4j

query = """
        // Load and commit every 500 records
        USING PERIODIC COMMIT 500 
        LOAD CSV WITH HEADERS FROM 'file:////home/igti/neo4j_instacart/import/neo4j_products.csv' AS line FIELDTERMINATOR '|' 
        WITH line 
                
        // Create Product, Aisle and Department nodes
        merge (product:Product {product_id: toInteger(line.product_id), product_name: line.product_name}) 
        MERGE  (aisle:Aisle {aisle_id: toInteger(line.aisle_id)}) 
        MERGE  (department:Department {department_id: toInteger(line.department_id)})      

        // Create relationships between products and aisles & products and departments 
        CREATE (product)-[:IN_AISLE]->(aisle) 
        CREATE (product)-[:IN_DEPARTMENT]->(department);
        
        """

g.run(query)



In [22]:
query2 = """
        // Aisle data is very small, so there is no need to do periodic commits
        LOAD CSV WITH HEADERS FROM 'file:////home/igti/neo4j_instacart/import/neo4j_aisles.csv' AS line FIELDTERMINATOR '|' 
        WITH line 
        
        // For each Aisle node, set property 'aisle_name' 
        MATCH (aisle:Aisle {aisle_id: toInteger(line.aisle_id)}) 
        SET aisle.aisle_name = line.aisle;
        """

g.run(query2)

In [23]:
query3 = """
        // Department data is very small, so there is no need to do periodic commits
        LOAD CSV WITH HEADERS FROM 'file:////home/igti/neo4j_instacart/import/neo4j_departments.csv' AS line FIELDTERMINATOR '|' 
        WITH line
        
        // For each Department node, set property 'department_name' 
        MATCH (department:Department {department_id: toInteger(line.department_id)}) 
        SET department.department_name = line.department;
        """

g.run(query3)

In [24]:
query4 = """
        // Load and commit every 500 records        
        USING PERIODIC COMMIT 500
        LOAD CSV WITH HEADERS FROM 'file:////home/igti/neo4j_instacart/import/neo4j_order_product.csv' AS line FIELDTERMINATOR '|'
        WITH line
        
        // Create Order nodes and then create relationships between orders and products
        MERGE (order:Order {order_id: toInteger(line.order_id)})
        MERGE (product:Product {product_id: toInteger(line.product_id)})
        CREATE (order)-[:CONTAINS]->(product);
        """

g.run(query4)

In [25]:
query5 = """
        // Load and commit every 500 records 
        USING PERIODIC COMMIT 500
        LOAD CSV WITH HEADERS FROM 'file:////home/igti/neo4j_instacart/import/neo4j_order_user.csv' AS line FIELDTERMINATOR '|'
        WITH line
        
        // Create User nodes and then create relationships between users and orders 
        MERGE (order:Order {order_id: toInteger(line.order_id)})
        MERGE (user:User   {user_id:  toInteger(line.user_id)})

        // Create relationships between users and orders, then set Order properties
        CREATE(user)-[o:ORDERED]->(order)              
        SET order.order_number = toInteger(line.order_number),
            order.order_day_of_week = toInteger(line.order_dow), 
            order.order_hour_of_day = toInteger(line.order_hour_of_day);
        """
g.run(query5)

In [26]:
# Implements user-user collaborative filtering using the following steps:
#   1. For each user pair, compute Jaccard index
#   2. For each target user, select top k neighbours based on Jaccard index
#   3. Identify products purchased by the top k neighbours that have not been purchased by the target user
#   4. Rank these products by the number of purchasing neighbours
#   5. Return the top n recommendations for each user

def collaborative_filtering(graph, neighbourhood_size, num_recos):

    query = """
           // Get user pairs and count of distinct products that they have both purchased
           MATCH (u1:User)-[:ORDERED]->(:Order)-[:CONTAINS]->(p:Product)<-[:CONTAINS]-(:Order)<-[:ORDERED]-(u2:User)
           WHERE u1 <> u2
           WITH u1, u2, COUNT(DISTINCT p) as intersection_count

           // Get count of all the distinct products that they have purchased between them
           MATCH (u:User)-[:ORDERED]->(:Order)-[:CONTAINS]->(p:Product)
           WHERE u in [u1, u2]
           WITH u1, u2, intersection_count, COUNT(DISTINCT p) as union_count

           // Compute Jaccard index
           WITH u1, u2, intersection_count, union_count, (intersection_count*1.0/union_count) as jaccard_index

           // Get top k neighbours based on Jaccard index
           ORDER BY jaccard_index DESC, u2.user_id
           WITH u1, COLLECT(u2)[0..{k}] as neighbours
           WHERE LENGTH(neighbours) = {k}                                              // only want users with enough neighbours
           UNWIND neighbours as neighbour
           WITH u1, neighbour

           // Get top n recommendations from the selected neighbours
           MATCH (neighbour)-[:ORDERED]->(:Order)-[:CONTAINS]->(p:Product)             // get all products bought by neighbour
           WHERE not (u1)-[:ORDERED]->(:Order)-[:CONTAINS]->(p)                        // which target user has not already bought
           WITH u1, p, COUNT(DISTINCT neighbour) as cnt                                // count neighbours who purchased product
           ORDER BY u1.user_id, cnt DESC                                               // sort by count desc
           RETURN u1.user_id as user, COLLECT(p.product_name)[0..{n}] as recos         // return top n products
           """

    recos = {}
    for row in graph.run(query, k=neighbourhood_size, n=num_recos):
        recos[row[0]] = row[1]

    return recos

In [31]:
%%time
recommendations = collaborative_filtering(g,10,1)
display(recommendations)

{701: ['Strawberries'],
 1562: ['Organic Whole String Cheese'],
 4789: ['Organic Granny Smith Apple'],
 5225: ['Bag of Organic Bananas'],
 5939: ['Organic Lemon'],
 6043: ['Organic Garlic'],
 6389: ['Organic Zucchini'],
 7968: ['Organic Strawberries'],
 12906: ['Bag of Organic Bananas'],
 24670: ['Organic Blueberries'],
 25442: ['Organic Blueberries'],
 25490: ['Bag of Organic Bananas'],
 26277: ['Bag of Organic Bananas'],
 32976: ['Bag of Organic Bananas'],
 37120: ['Organic Strawberries'],
 40286: ['Bag of Organic Bananas'],
 42145: ['Organic Baby Spinach'],
 43902: ['Large Lemon'],
 45067: ['Organic Yellow Onion'],
 47838: ['Organic Lacinato (Dinosaur) Kale'],
 49441: ['Organic Yellow Onion'],
 50241: ['Yellow Onions'],
 51076: ['Red Onion'],
 52784: ['Bag of Organic Bananas'],
 53304: ['Organic Baby Carrots'],
 53968: ['Strawberries'],
 55720: ['Fresh Cauliflower'],
 56266: ['Banana'],
 58959: ['Organic Unsweetened Almond Milk'],
 59889: ['Bag of Organic Bananas'],
 61065: ['Organi

CPU times: user 58.9 ms, sys: 987 µs, total: 59.9 ms
Wall time: 1min 55s


In [28]:
recommendations[4789]

['Organic Blackberries', 'Organic Granny Smith Apple']

In [29]:


query = """
        // Get count of all distinct products that user 4789 has purchased and find other users who have purchased them
        MATCH (u1:User)-[:ORDERED]->(:Order)-[:CONTAINS]->(p:Product)<-[:CONTAINS]-(:Order)<-[:ORDERED]-(u2:User)
        WHERE u1 <> u2
          AND u1.user_id = {uid}
        WITH u1, u2, COUNT(DISTINCT p) as intersection_count
        
        // Get count of all the distinct products that are unique to each user
        MATCH (u:User)-[:ORDERED]->(:Order)-[:CONTAINS]->(p:Product)
        WHERE u in [u1, u2]
        WITH u1, u2, intersection_count, COUNT(DISTINCT p) as union_count
       
        // Compute Jaccard index
        WITH u1, u2, intersection_count, union_count, (intersection_count*1.0/union_count) as jaccard_index
        
        // Get top k neighbours based on Jaccard index
        ORDER BY jaccard_index DESC, u2.user_id
        WITH u1, COLLECT([u2.user_id, jaccard_index, intersection_count, union_count])[0..{k}] as neighbours
     
        WHERE LENGTH(neighbours) = {k}                // only want to return users with enough neighbours
        RETURN u1.user_id as user, neighbours
        """

neighbours = {}
for row in g.run(query, uid=4789, k=10):
    neighbours[row[0]] = row[1]

print("Labels for user 4789's neighbour list: user_id, jaccard_index, intersection_count, union count")
display(neighbours)



Labels for user 4789's neighbour list: user_id, jaccard_index, intersection_count, union count


{4789: [[42145, 0.12794612794612795, 38, 297],
  [138203, 0.10497237569060773, 38, 362],
  [87350, 0.09390862944162437, 37, 394],
  [49441, 0.0912280701754386, 26, 285],
  [187754, 0.0912280701754386, 26, 285],
  [180461, 0.09115281501340483, 34, 373],
  [120660, 0.08641975308641975, 21, 243],
  [107931, 0.08360128617363344, 26, 311],
  [73477, 0.07855626326963906, 37, 471],
  [154852, 0.0735930735930736, 17, 231]]}

In [30]:
%%time
query = """
        // Get top n recommendations for user 4789 from the selected neighbours
        MATCH (u1:User),
              (neighbour:User)-[:ORDERED]->(:Order)-[:CONTAINS]->(p:Product)        // get all products bought by neighbour
        WHERE u1.user_id = {uid}
          AND neighbour.user_id in {neighbours}
          AND not (u1)-[:ORDERED]->(:Order)-[:CONTAINS]->(p)                        // which u1 has not already bought
        
        WITH u1, p, COUNT(DISTINCT neighbour) as cnt                                // count times purchased by neighbours
        ORDER BY u1.user_id, cnt DESC                                               // and sort by count desc
        RETURN u1.user_id as user, COLLECT([p.product_name,cnt])[0..{n}] as recos  
        """

recos = {}
for row in g.run(query, uid=4789, neighbours=[42145,138203,87350,49441,187754,180461,120660,107931,73477,154852], n=10):
    recos[row[0]] = row[1]
    
print("Labels for user 4789's recommendations list: product, number of purchasing neighbours")
display(recos)

Labels for user 4789's recommendations list: product, number of purchasing neighbours


{4789: [['Organic Granny Smith Apple', 6],
  ['Organic Cilantro', 5],
  ['Limes', 5],
  ['Organic Green Cabbage', 5],
  ['Corn Tortillas', 5],
  ['Organic Grape Tomatoes', 5],
  ['Creamy Almond Butter', 5],
  ['Organic Sugar Snap Peas', 4],
  ['Organic Lacinato (Dinosaur) Kale', 4],
  ['Organic Blackberries', 4]]}

CPU times: user 6.54 ms, sys: 4.62 ms, total: 11.2 ms
Wall time: 4.57 s
